# Azure AI Foundry Agent with Microsoft Learn MCP

This notebook demonstrates how to create an Azure AI Foundry agent that can search and fetch Microsoft Learn documentation using the Model Context Protocol (MCP).

## Import the Needed Packages

We'll import all the necessary libraries for creating our Azure AI Foundry agent with MCP support.

In [ ]:
# Install required packages from requirements.txt
%pip install -r requirements-aifoundry.txt

In [ ]:
# Import necessary libraries - Following official Microsoft documentation
import os, time
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential
from azure.ai.agents.models import (
    ListSortOrder,
    McpTool,
    RequiredMcpToolCall,
    RunStepActivityDetails,
    SubmitToolApprovalAction,
    ToolApproval,
)

from IPython.display import display, HTML
import requests
from dotenv import load_dotenv

## Setting up Azure AI Foundry Connection

Configure the Azure AI Foundry project connection and MCP server settings.

In [ ]:
load_dotenv()

# Get MCP server configuration from environment variables - Following Microsoft docs
mcp_server_url = os.environ.get("MCP_SERVER_URL", "https://learn.microsoft.com/api/mcp")
mcp_server_label = os.environ.get("MCP_SERVER_LABEL", "mslearn")

# Azure AI Foundry project configuration
PROJECT_ENDPOINT = os.environ.get("PROJECT_ENDPOINT", os.environ.get("AZURE_AI_AGENT_ENDPOINT"))

# Override with valid model deployment name
MODEL_DEPLOYMENT_NAME = os.environ.get("AZURE_AI_MODEL_NAME", "gpt-4o")  # Use standard gpt-4o model

print(f"MCP Server URL: {mcp_server_url}")
print(f"MCP Server Label: {mcp_server_label}")
print(f"Project Endpoint: {PROJECT_ENDPOINT}")
print(f"Model Deployment: {MODEL_DEPLOYMENT_NAME}")

## Creating the Azure AI Foundry Agent

Create an Azure AI Foundry agent with access to Microsoft Learn documentation through MCP tools.

In [ ]:
# Initialize Azure AI Project client - Following Microsoft documentation
project_client = AIProjectClient(
    endpoint=PROJECT_ENDPOINT,
    credential=DefaultAzureCredential(),
)

# Initialize agent MCP tool for Microsoft Learn
mcp_tool = McpTool(
    server_label=mcp_server_label,
    server_url=mcp_server_url,
    allowed_tools=[],  # Optional: specify allowed tools - empty allows all
)

# You can also add or remove allowed tools dynamically
microsoft_docs_search = "microsoft_docs_search"
microsoft_docs_fetch = "microsoft_docs_fetch"
mcp_tool.allow_tool(microsoft_docs_search)
mcp_tool.allow_tool(microsoft_docs_fetch)
mcp_tool.set_approval_mode("never")
print(f"Allowed tools: {mcp_tool.allowed_tools}")

# Create agent with MCP tool
with project_client:
    agents_client = project_client.agents

    # Create a new agent
    agent = agents_client.create_agent(
        model=MODEL_DEPLOYMENT_NAME,
        name="Microsoft Learn MCP Agent",
        instructions="""You are a helpful agent that can use Microsoft Learn MCP tools to assist users. 

Use the available MCP tools to search and fetch Microsoft Learn documentation:
- microsoft_docs_search: Search Microsoft documentation for specific topics
- microsoft_docs_fetch: Retrieve specific documentation content

When helping users:
- Search for relevant documentation using microsoft_docs_search
- Fetch detailed content when users need specific information
- Provide direct links to documentation when possible
- Summarize complex topics clearly

Always verify information by checking the official Microsoft documentation through the MCP tools.""",
        tools=mcp_tool.definitions,
    )

    print(f"✅ Created agent, ID: {agent.id}")
    print(f"Agent Name: {agent.name}")
    print(f"MCP Server: {mcp_tool.server_label} at {mcp_tool.server_url}")
    print(f"Tools configured: {len(mcp_tool.definitions)}")
    
    # Store agent ID for later use
    global current_agent_id
    current_agent_id = agent.id

## Agent Interaction Function

Create a function to handle user interactions with the agent, displaying the conversation and tool usage.

In [ ]:
async def chat_with_agent(user_input, thread_id=None):
    """Chat with the Azure AI Foundry agent using MCP tools."""
    
    # Create new client for this interaction
    client = AIProjectClient(
        endpoint=PROJECT_ENDPOINT,
        credential=DefaultAzureCredential(),
    )
    
    with client:
        agents_client = client.agents
        
        # Use existing thread or create new one
        if thread_id:
            thread = agents_client.threads.get(thread_id)
        else:
            thread = agents_client.threads.create()
            
        print(f"🗨️ User: {user_input}")
        
        # Create message
        message = agents_client.messages.create(
            thread_id=thread.id,
            role="user",
            content=user_input
        )
        
        # Create run with MCP tools
        run = agents_client.runs.create(
            thread_id=thread.id,
            agent_id=current_agent_id,
            tool_resources=mcp_tool.resources
        )
        
        # Process run until completion
        while run.status in ["queued", "in_progress", "requires_action"]:
            time.sleep(2)
            run = agents_client.runs.get(thread_id=thread.id, run_id=run.id)
            
            if run.status == "requires_action" and isinstance(run.required_action, SubmitToolApprovalAction):
                tool_calls = run.required_action.submit_tool_approval.tool_calls
                if tool_calls:
                    tool_approvals = []
                    for tool_call in tool_calls:
                        if isinstance(tool_call, RequiredMcpToolCall):
                            print(f"🔧 Using tool: {tool_call.function.name}")
                            tool_approvals.append(
                                ToolApproval(
                                    tool_call_id=tool_call.id,
                                    approve=True,
                                    headers=mcp_tool.headers,
                                )
                            )
                    
                    if tool_approvals:
                        agents_client.runs.submit_tool_outputs(
                            thread_id=thread.id,
                            run_id=run.id,
                            tool_approvals=tool_approvals
                        )
        
        # Display response
        if run.status == "completed":
            messages = agents_client.messages.list(thread_id=thread.id, order=ListSortOrder.DESCENDING)
            for msg in messages:
                if msg.role == "assistant" and msg.text_messages:
                    last_text = msg.text_messages[-1]
                    print(f"🤖 Assistant: {last_text.text.value}")
                    break
        else:
            print(f"❌ Run failed with status: {run.status}")
            
        return thread.id

print("✅ Chat function created successfully!")

## Testing the Agent

Let's test our agent with questions about Microsoft technologies and Azure services.

In [ ]:
# Test the agent with a question about Azure AI Foundry
test_question = "How do I get started with Azure AI Foundry? Please search Microsoft Learn documentation."

print("Testing the Azure AI Foundry agent with MCP tools...")
thread_id = await chat_with_agent(test_question)

## Cleanup

Clean up the Azure AI Foundry agent when done.

In [ ]:
# Clean up the agent when finished
try:
    cleanup_client = AIProjectClient(
        endpoint=PROJECT_ENDPOINT,
        credential=DefaultAzureCredential(),
    )
    
    with cleanup_client:
        agents_client = cleanup_client.agents
        agents_client.delete_agent(current_agent_id)
        print(f"🗑️ Deleted agent: {current_agent_id}")
    
except Exception as e:
    print(f"⚠️ Cleanup error: {e}")

print("\n🎉 Azure AI Foundry + Microsoft Learn MCP session completed!")
print("? Documentation: https://learn.microsoft.com/en-us/azure/ai-foundry/agents/how-to/tools/model-context-protocol-samples")